In [1]:
# Processing pipeline to turn generated data into a format that is easy to read into tensorflow: TFRecord files.
# These files allow for fast read and write to Tensorflow architectures, and are preferred using the new Dataset API. 
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import imageio
import numpy as np
import glob
from skimage.transform import resize
import sys

# The documentation for this code is somewhat spotty, and has been cobbled together from a variety of sources:
# https://medium.com/@WuStangDan/step-by-step-tensorflow-object-detection-api-tutorial-part-2-converting-dataset-to-tfrecord-47f24be9248d
# http://machinelearninguru.com/deep_learning/tensorflow/basics/tfrecord/tfrecord.html
# https://www.skcript.com/svr/why-every-tensorflow-developer-should-know-about-tfrecord/
# tensorflow/inception/inception/data/build_image_Data.py

# Once within the format of tfrecord files, it is easier to combine datasets, to split them, etc, so we just treat
# these on a video by video basis. 


In [2]:
## Designate helper function to define features for examples more easily
def _int64_feature_(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature_(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def _float_feature_(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _full_example_(videoname,reader,i):
    resize_param = 128
    img_0 = reader.get_data(i)
    img_0_ds = resize(img_0,(resize_param,resize_param))
    

    features = {'video': _bytes_feature_(tf.compat.as_bytes((videoname))),
                'frame': _int64_feature_(i),
                'image': _bytes_feature_(tf.compat.as_bytes(img_0_ds.tostring())),
                }
    example = tf.train.Example(features = tf.train.Features(feature = features))
    return example

def _encoder_example_b_(videoname,reader,i,params):
    resize_param = 128
    img_0 = reader.get_data(i)
    img_0_ds = resize(img_0,(resize_param,resize_param))
    intensity = img_0_ds[int(resize_param/2),int(resize_param/2),0]
    print(intensity)
    features = {'video': _bytes_feature_(tf.compat.as_bytes((videoname))),
                'frame': _int64_feature_(i),
                'image': _bytes_feature_(tf.compat.as_bytes(img_0_ds.tostring())),
                'intensity':_float_feature_(intensity),
                'background':_bytes_feature_(tf.compat.as_bytes(params.tostring()))
                }
    example = tf.train.Example(features = tf.train.Features(feature = features))
    return example

def _encoder_example_b_m_(videoname,reader,i,params,pos):
    resize_param = 128
    img_0 = reader.get_data(i)
    img_0_ds = resize(img_0,(resize_param,resize_param))
    intensity = img_0_ds[int(resize_param/2),int(resize_param/2),0]
    print(intensity)
    features = {'video': _bytes_feature_(tf.compat.as_bytes((videoname))),
                'frame': _int64_feature_(i),
                'image': _bytes_feature_(tf.compat.as_bytes(img_0_ds.tostring())),
                'intensity':_float_feature_(intensity),
                'background':_bytes_feature_(tf.compat.as_bytes(params.tostring())),
                'position':_bytes_feature_(tf.compat.as_bytes(pos.tostring()))
                }
    example = tf.train.Example(features = tf.train.Features(feature = features))
    return example

def _encoder_example_eff_(videoname,reader,i,params):
    resize_param = 128
    img_0 = reader.get_data(i)
    img_0_ds = resize(img_0,(resize_param,resize_param))
    intensity = img_0_ds[int(resize_param/2),int(resize_param/2),0]
    print(intensity)
    features = {'video': _bytes_feature_(tf.compat.as_bytes((videoname))),
                'frame': _int64_feature_(i),
                'image': _bytes_feature_(tf.compat.as_bytes(img_0_ds.tostring())),
                'background':_bytes_feature_(tf.compat.as_bytes(params.tostring()))
                }
    example = tf.train.Example(features = tf.train.Features(feature = features))
    return example

In [23]:
## To generate a tfrecords file for full training

def vids_to_fulltrain(datadirectory,extension):
    # We iterate through all directories
    videos = glob.glob(datadirectory)
    for videoname in videos:
        print('working on video: ' +videoname)
        # Choose a video, and load it
        video = imageio.get_reader(videoname,'ffmpeg')
        # We want to open a corresponding tfRecords file
        datafile_name = videoname.split('.mp4')[0]+extension
        writer = tf.python_io.TFRecordWriter(datafile_name)
        nb_frames = video.get_length()
        for i in range(nb_frames):
            # Iterate through frames
            example = _full_example_(videoname,video,i)
            writer.write(example.SerializeToString())
        writer.close()
        sys.stdout.flush()

datadirectory = 'datadirectory/toydynamics_nograv/Video_ball_color_small_b9.mp4'
extension = 'train.tfrecords'
vids_to_fulltrain(datadirectory,extension)

working on video: datadirectory/toydynamics_nograv/Video_ball_color_small_b9.mp4


/Users/taigaabe/anaconda3/envs/constrainedgen/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [12]:
## To generate a tfrecords file for encoder only (VIN) training
datadirectory = 'datadirectory/toydynamics_nograv/*.mp4'
extension = 'encoder_train.tfrecords'

def vids_to_encodertrain(datadirectory,extension): ## assumes the ball is in the center 
    # We iterate through all directories
    videos = glob.glob(datadirectory)
    for videoname in videos:
        print('working on video: ' +videoname)
        # Choose a video, and load it
        video = imageio.get_reader(videoname,'ffmpeg')
        # We want to open a corresponding tfRecords file
        datafile_name = videoname.split('.mp4')[0]+extension
        writer = tf.python_io.TFRecordWriter(datafile_name)
        nb_frames = video.get_length()
        for i in range(nb_frames):
            # Iterate through frames
            example = _encoder_example_(videoname,video,i)
            writer.write(example.SerializeToString())
        writer.close()
        sys.stdout.flush()

working on video: datadirectory/toydynamics_nograv/Video_ball_color_corrback.mp4
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887


/Users/taigaabe/anaconda3/envs/constrainedgen/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648

0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633

0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633

...

0.9372549019607843
0.9058823529411765
0.8588235294117645
0.807843137254902
0.7529411764705882
0.6941176470588235
0.6313725490196078
0.5725490196078431
0.5019607843137255
0.43529411764705883
0.3686274509803922
0.3058823529411765
0.24705882352941178
0.19215686274509808
0.1450980392156863
0.10196078431372549
0.06274509803921569
0.03529411764705882
0.011764705882352941
0.00392156862745098
0.0
0.00392156862745098
0.0196078431372549
0.0392156862745098
0.07450980392156863
0.10980392156862745
0.1568627450980392
0.2117647058823529
0.26666666666666666
0.3215686274509804
0.3882352941176471
0.4509803921568627
0.5176470588235295
0.5843137254901961
0.6470588235294118
0.7058823529411765
0.7686274509803923
0.8156862745098039
0.8705882352941177
0.9098039215686275
0.9450980392156862
0.9725490196078431
0.9882352941176471
1.0
1.0
0.9921568627450981
0.9764705882352942
0.9490196078431372
0.9176470588235295
0.8784313725490196
0.8313725490196079
0.7764705882352942
0.7215686274509804
0.6588235294117647
0.6
0.5

0.2117647058823529
0.1568627450980392
0.11372549019607844
0.07450980392156863
0.0392156862745098
0.0196078431372549
0.00784313725490196
0.0
0.00392156862745098
0.011764705882352941
0.03529411764705882
0.06274509803921569
0.09411764705882353
0.1411764705882353
0.19215686274509808
0.24705882352941178
0.3058823529411765
0.36470588235294116
0.4274509803921569
0.4980392156862745
0.5647058823529412
0.6274509803921569
0.6901960784313725
0.7490196078431373
0.803921568627451
0.8549019607843138
0.8980392156862744
0.9333333333333333
0.9647058823529413
0.9882352941176471
0.996078431372549
1.0
0.996078431372549
0.9803921568627451
0.9607843137254903
0.9333333333333333
0.8901960784313725
0.8509803921568626
0.796078431372549
0.7333333333333333
0.6784313725490196
0.615686274509804
0.5490196078431373
0.48235294117647065
0.4196078431372549
0.35294117647058826
0.29411764705882354
0.2392156862745098
0.1843137254901961
0.13333333333333333
0.09019607843137255
0.054901960784313725
0.03137254901960784
0.011764

working on video: datadirectory/toydynamics_nograv/Video_ball_color_dynback.mp4
0.9921568627450981
0.9921568627450981
0.9921568627450981
0.9607843137254903
0.9254901960784314
0.9058823529411765
0.8666666666666667
0.8
0.7490196078431373
0.6901960784313724
0.6374560786228554
0.5611606373506433
0.48235294117647065
0.43094566195618883
0.3834844851026348
0.3067881864659927
0.24884446088005527
0.19135365205652588
0.13984374999999996
0.11505821078431377
0.08235294117647059
0.0392156862745098
0.017018995098039245
0.00784313725490196
0.00784313725490196
0.022196691176470558
0.0196078431372549
0.027450980392156862
0.058823529411764705
0.0784313725490196
0.1607843137254902
0.20696333342907475
0.26659007352941166
0.3315737256816789
0.392156862745098
0.4666666666666667
0.5254901960784314
0.588082107843137
0.6431372549019608
0.7058823529411765
0.803921568627451
0.8313725490196079
0.8549019607843138
0.9425091911764707
0.9699601715686275
0.9803921568627451
1.0
1.0
0.984313725490196
0.9812719606885723


0.2
0.1450980392156863
0.10196078431372549
0.058823529411764705
0.03529411764705882
0.010935465494791775
0.0
0.0
0.0
0.0
0.054901960784313725
0.07450980392156863
0.12156862745098039
0.17647058823529413
0.20517769607843134
0.28619791666666655
0.324234068627451
0.3908241421568627
0.4766717648973651
0.5371783088235293
0.5995731129365808
0.6653339460784313
0.7289588479434742
0.784313725490196
0.8745098039215686
0.8823529411764706
0.9137254901960783
0.9607843137254903
0.9712928921568628
1.0
0.9773503920611214
0.9712928921568628
0.9712928921568628
0.9687040441176472
0.9468576468673406
0.9171941420611214
0.8539455638212314
0.8191549263748468
0.7672947303921569
0.6902726715686276
0.615686274509804
0.5647058823529412
0.512469362745098
0.45349264705882336
0.38861161774280006
0.324157475490196
0.2531163234336703
0.20909926470588225
0.1612372304878983
0.10633526970358456
0.08235294117647059
0.03137254901960784
0.0392156862745098
0.00784313725490196
0.00392156862745098
0.00784313725490196
0.0126445

0.4235294117647058
0.388235294117647
0.29411764705882354
0.25138274548100487
0.2100556616689645
0.14901960784313725
0.09856872558593766
0.0784313725490196
0.0392156862745098
0.011764705882352941
0.00784313725490196
0.019684436274509887
0.01568627450980392
0.012217622644761058
0.0196078431372549
0.054901960784313725
0.10196078431372549
0.1411764705882353
0.21168811274509788
0.2536458333333334
0.3437653186274509
0.3764705882352941
0.46309539196537974
0.52410691205193
0.5738817401960784
0.6326286764705882
0.6941176470588235
0.7686274509803923
0.803921568627451
0.8784313725490196
0.9019607843137254
0.9333333333333333
0.9647058823529413
1.0
1.0
1.0
0.9817248774509804
0.9882352941176471
0.9764705882352942
0.9498228185317092
0.8862745098039215
0.8392156862745098
0.7711657355813417
0.7232776716643688
0.6444699754901961
0.6326547061695771
0.5368019851983762
0.472220686370251
0.41443014705882364
0.3451240689146753
0.2784313725490196
0.21910436293658078
0.1607843137254902
0.13725490196078433
0.07

In [18]:
## To generate a tfrecords file for encoder only (VIN) training

video_title = 'Video_ball_color_small'
loadmat = np.load(video_title+'loadmat.npy')
params = np.load(video_title+'params.npy')

datadirectory = 'datadirectory/toydynamics_nograv/'+video_title+'_*.mp4'
extension = 'encoder_train.tfrecords'



def vids_to_encodertrain_b(datadirectory,extension,loadmat,params): ## assumes the ball is in the center 
    # We iterate through all directories
    videos = glob.glob(datadirectory)
    print(videos)
    for videoname in videos:
        print('working on video: ' +videoname)
        # Choose a video, and load it
        video = imageio.get_reader(videoname,'ffmpeg')
        # We want to open a corresponding tfRecords file
        datafile = videoname.split('.mp4')[0]
        datafile_name = datafile+extension
        number = int(datafile.split('Video_ball_color_small_b')[1])
        print(number)
        writer = tf.python_io.TFRecordWriter(datafile_name)
        nb_frames = video.get_length()
        params_write = params[number,:]
        for i in range(nb_frames):
            # Iterate through frames
            example = _encoder_example_b_(videoname,video,i,params_write)
            writer.write(example.SerializeToString())
        writer.close()
        sys.stdout.flush()
vids_to_encodertrain_p(datadirectory,extension,loadmat,params)

['datadirectory/toydynamics_nograv/Video_ball_color_small_b9.mp4', 'datadirectory/toydynamics_nograv/Video_ball_color_small_b8.mp4', 'datadirectory/toydynamics_nograv/Video_ball_color_small_b19.mp4', 'datadirectory/toydynamics_nograv/Video_ball_color_small_b18.mp4', 'datadirectory/toydynamics_nograv/Video_ball_color_small_b6.mp4', 'datadirectory/toydynamics_nograv/Video_ball_color_small_b15.mp4', 'datadirectory/toydynamics_nograv/Video_ball_color_small_b14.mp4', 'datadirectory/toydynamics_nograv/Video_ball_color_small_b7.mp4', 'datadirectory/toydynamics_nograv/Video_ball_color_small_b5.mp4', 'datadirectory/toydynamics_nograv/Video_ball_color_small_b16.mp4', 'datadirectory/toydynamics_nograv/Video_ball_color_small_b17.mp4', 'datadirectory/toydynamics_nograv/Video_ball_color_small_b4.mp4', 'datadirectory/toydynamics_nograv/Video_ball_color_small_b0.mp4', 'datadirectory/toydynamics_nograv/Video_ball_color_small_b13.mp4', 'datadirectory/toydynamics_nograv/Video_ball_color_small_b12.mp4', '

/Users/taigaabe/anaconda3/envs/constrainedgen/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


0.9933624267578123
0.9933624267578123
0.9659114463656555
0.9328298531326593
0.8874800738166359
0.8465553433287376
0.78426316205193
0.7293612012676164
0.671870392444087
0.6143535539215687
0.5476868872549019
0.4753896077473959
0.41568627450980394
0.34901960784313724
0.28715431362974875
0.2244092155905331
0.16950725480621937
0.12636999990425857
0.08323274500229778
0.04881843118106617
0.021367450788909313
0.0008798038258272054
0.0008798038258272054
0.0008798038258272054
0.0008798038258272054
0.0008798038258272054
0.028330784217984066
0.0636249018650429
0.09891901951210172
0.14597784304151348
0.2008798038258272
0.2557817646101409
0.31727073519837634
0.39474571078431364
0.45578176461014097
0.5185268626493567
0.5843137254901961
0.6470588235294118
0.7089241177428003
0.7747109805836399
0.8187280393114278
0.8697084314682906
0.9089241177428004
0.9433384315640319
0.963826078527114
0.9834339216643688
0.9834339216643688
0.9834339216643688
0.9755907844094669
0.963826078527114
0.9433384315640319
0.901

0.1023110782398895
0.07219465667126194
0.0408221076516541
0.0408221076516541
0.0408221076516541
0.0408221076516541
0.0408221076516541
0.0408221076516541
0.06435151941635998
0.09572406843596783
0.13752860274969325
0.17807700961243844
0.24082210765165413
0.29054637235753655
0.35550399481081485
0.42129085765165425
0.48403595569087
0.5467810537300857
0.6016830145143994
0.6774749755859374
0.7341365440219055
0.7977614458869484
0.8469822303921567
0.8952971813725488
0.9345128676470585
0.9624933579388786
0.9899443383310353
0.9899443383310353
0.9899443383310353
0.9899443383310353
0.9899443383310353
0.9899443383310353
0.9541973039215684
0.9228247549019606
0.8744332107843136
0.8234528186274508
0.7738051470588234
0.7254136029411763
0.6566875981349571
0.5874320834290749
0.5254901960784315
0.4583966423483456
0.39386590695848644
0.3324535295075061
0.2692555147058823
0.20909926470588228
0.1607843137254902
0.10980392156862745
0.07450980392156863
0.05919985303691784
0.03567044127221196
0.0204370835248161

0.7728993135340072
0.7075148040173098
0.6412750244140624
0.5776501225490194
0.5096001718558514
0.4421042648016235
0.38461345597809415
0.324080882352941
0.26525735294117625
0.2021359312768075
0.1537443871591604
0.10756536745557585
0.07619281843596802
0.04394046559053295
0.027424950693167832
0.013071396771599235
0.013071396771599235
0.013071396771599235
0.029133994906556264
0.04394046559053295
0.07619281843596802
0.11586142147288594
0.1624673282398896
0.20864634794347414
0.272271249808517
0.3271732105928308
0.3911744360830267
0.45917382333792883
0.5227987252029718
0.5859967400045956
0.6483655144186582
0.7136994604970894
0.760381960401348
0.8152839211856618
0.8653845095166974
0.9080688476562501
0.9290354410807294
0.9525648528454351
0.9712928921568629
0.9712928921568629
0.9712928921568629
0.9712928921568629
0.9712928921568629
0.9399203431372551
0.9015844606885725
0.8662903430415136
0.8301164215686276
0.7752144607843139
0.715511127546722
0.6549019607843138
0.5904478185317097
0.5277027204924

...

0.11460529402190567
0.07538960774739586
0.052739999808517214
0.036173921472886054
0.036173921472886054
0.036173921472886054
0.044017058727788015
0.052739999808517214
0.07538960774739586
0.11460529402190567
0.14989941166896448
0.2087229410807292
0.2675464704924939
0.32244843127680767
0.3812719606885724
0.43617392147288603
0.4997988233379289
0.5582720588235295
0.6223498774509806
0.6912290685317098
0.7450980392156863
0.8
0.8627450980392157
0.8989956126493568
0.9368785783356313
0.9694321576286764
0.9890400007659312
0.9982403923483456
0.9982403923483456
0.9982403923483456
0.9929615693933822
0.9694321576286764
0.9528660792930453
0.9262949027267156
0.8665915694891237
0.8134492163564644
0.760306863223805
0.708044313916973
0.6496982350068934
0.5829810049019608
0.5202359068627451
0.43529411764705883
0.36470588235294116
0.30980392156862746
0.25098039215686274
0.18823529411764706
0.1411764705882353
0.09150426528033101
0.06840024461933203
0.03702769559972418
0.0331061269722732
0.013498283835018294


0.048844460880055285
0.026194852941176607
0.007466813629748941
0.007466813629748941
0.007466813629748941
0.015762867647059017
0.026194852941176607
0.06236877441406266
0.09811580882352962
0.13599877450980408
0.1804687500000002
0.24100132362515336
0.3046262254901962
0.3660386029411765
0.4283568139169731
0.4897691913679534
0.5546502206839767
0.6230259315640317
0.6810968137254902
0.7395440195121019
0.795325784122243
0.8465813730277267
0.895399804208793
0.9337356866574755
0.9611866670496323
0.9773503920611214
1.0
1.0
1.0
0.9795123530369179
0.9611866670496323
0.9337356866574755
0.9014833338120404
0.8513827454810049
0.8012821571499693
0.7463801963656556
0.687932990579044
0.6291094611672792
0.5699096081303615
0.49715941186044726
0.4335345099954043
0.3581954656862746
0.30329350490196083
0.24230801451439957
0.1913276223575368
0.14117647058823526
0.10588235294117644
0.06674325980392162
0.03929227941176476
0.01576286764705888
0.01576286764705888
0.01576286764705888
0.01576286764705888
0.0301164215

0.9624933579388787
0.9389639461741728
0.9115129657820158
0.8762188481349571
0.8291600246055453
0.7738817401960785
0.7137254901960784
0.656234681372549
0.5965313481349571
0.5303175982306986
0.4710917154947918
0.40965330834482233
0.34607896991804543
0.28509347953048414
0.22707316080729156
0.17654568541283694
0.12727433746936262
0.07903597962622565
0.04550146963082126
0.02413401884191193
0.011036592371323636
0.011036592371323636
0.011036592371323636
0.011036592371323636
0.011036592371323636
0.04550146963082126
0.06903088139552715
0.11433009727328447
0.16093600404028813
0.20628578335631148
0.2707399256089156
0.3300163717830884
0.3927614698223041
0.452464803059896
0.5198841169768689
0.5852180630553002
0.6558583577473958
0.7188526228362437
0.7698330149931063
0.8187280393114276
0.8688286276424633
0.9089241177428002
0.9363750981349572
0.9656607833563112
0.9799898035386028
0.9860733331418503
0.9899949017693015
0.9799898035386028
0.9656607833563112
0.9382098029641543
0.9107588225719974
0.8636999

0.8963807349111521
0.9412270340265012
0.9686780144186582
0.9882858575559131
1.0
1.0
1.0
1.0
0.9686780144186582
0.9412270340265012
0.9098544850068935
0.8571650486366422
0.8022630878523285
0.7473611270680147
0.6924591662837011
0.6296635048062195
0.5659880395029105
0.504072181851256
0.43906399595971185
0.36676671645220577
0.3006280637254901
0.24313725490196075
0.19082414215686272
0.14901960784313723
0.10196078431372546
0.0745098039215686
0.043137254901960756
0.023529411764705854
0.019607843137254874
0.019607843137254874
0.019607843137254874
0.039215686274509776
0.05098039215686272
0.08323274500229777
0.12549019607843134
0.17166921578201588
0.21568627450980388
0.2793111763748468
0.3450980392156862
0.40784313725490196
0.4766717648973651
0.5402966667624081
0.6056306128408394
0.6700847550934435
0.7166906618604472
0.771592622644761
0.8130974264705884
0.863624901865043
0.9098039215686275
0.9464307598039217
0.9621170343137255
0.9778033088235295
0.9778033088235295
0.9738817401960784
0.96737132352

0.6784313725490196
0.6196078431372549
0.5529411764705883
0.4805182363472733
0.4199596330231311
0.3589741426355698
0.2979886522480085
0.23655024509803915
0.18164828431372543
0.132000612745098
0.08886335784313723
0.05623468137254904
0.028783700980392183
0.024862132352941203
0.024862132352941203
0.024862132352941203
0.024862132352941203
0.024862132352941203
0.04839154411764708
0.0836856617647059
0.11897977941176474
0.1660386029411765
0.21309742647058824
0.280190980200674
0.34469568589154415
0.4139512005974264
0.4784559062882965
0.5403212005016851
0.6052282595166972
0.6688531613817401
0.7324780632467831
0.7808450736251531
0.8274509803921568
0.8666666666666667
0.9106837253944546
0.9498994116689645
0.9695072548062194
0.9891150979434742
0.9891150979434742
0.9891150979434742
0.9773503920611214
0.9695072548062194
0.9459778430415136
0.899798823337929
0.867546470492494
0.8108849020565257
0.7528906130323223
0.6944694369446996
0.6338862998812805
0.5663398293887867
0.5061835793887867
0.4470588235294

In [22]:
## To generate a tfrecords file for encoder only (VIN) training

video_title = 'Video_ball_color_med_motion_w'
loadmat = np.load(video_title+'loadmat.npy')
params = np.load(video_title+'params.npy')
pos = np.load(video_title+'pos.npy')
index = np.linspace(-19.5,19.5,50)
yspace = np.linspace(-19.5,19.5,50)

xfull = np.tile(index,50).reshape(2500,1)
yfull = np.repeat(index,50).reshape(2500,1)

fullpos = np.concatenate((xfull,yfull),axis = 1)
pos = fullpos

frames = np.linspace(0,2500*100-1,30*100).astype('int')

datadirectory = 'datadirectory/toydynamics_nograv/'+'Video_ball_color_singleback_checktest3'+'_*.mp4'
extension = 'encoder_train.tfrecords'



def vids_to_encodertrain_b_m_(datadirectory,extension,loadmat,params,pos): ## assumes the ball is in the center 
    # We iterate through all directories
    videos = glob.glob(datadirectory)
    print(videos)
    for videoname in videos:
        print('working on video: ' +videoname)
        # Choose a video, and load it
        video = imageio.get_reader(videoname,'ffmpeg')
        # We want to open a corresponding tfRecords file
        datafile = videoname.split('.mp4')[0]
        datafile_name = datafile+extension
#         number = int(datafile.split(video_title+'test_b')[1])
#         print(number)
        writer = tf.python_io.TFRecordWriter(datafile_name)
        nb_frames = video.get_length()
        params_write = params[0,:2]
        for i in range(nb_frames):
            print(pos[i,:])
            # Iterate through frames
            example = _encoder_example_b_m_(videoname,video,i,params_write,pos[i,:])
            writer.write(example.SerializeToString())
        writer.close()
        sys.stdout.flush()
vids_to_encodertrain_b_m_(datadirectory,extension,loadmat,params,pos)

['datadirectory/toydynamics_nograv/Video_ball_color_singleback_checktest3_b0.mp4']
working on video: datadirectory/toydynamics_nograv/Video_ball_color_singleback_checktest3_b0.mp4
[-19.5 -19.5]
0.5659114463656555
[-18.70408163 -19.5       ]


/Users/taigaabe/anaconda3/envs/constrainedgen/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


0.5659114463656555
[-17.90816327 -19.5       ]
0.5659114463656555
[-17.1122449 -19.5      ]
0.5659114463656555
[-16.31632653 -19.5       ]
0.5659114463656555
[-15.52040816 -19.5       ]
0.5659114463656555
[-14.7244898 -19.5      ]
0.5659114463656555
[-13.92857143 -19.5       ]
0.5659114463656555
[-13.13265306 -19.5       ]
0.5659114463656555
[-12.33673469 -19.5       ]
0.5659114463656555
[-11.54081633 -19.5       ]
0.5659114463656555
[-10.74489796 -19.5       ]
0.5659114463656555
[ -9.94897959 -19.5       ]
0.5659114463656555
[ -9.15306122 -19.5       ]
0.5659114463656555
[ -8.35714286 -19.5       ]
0.5659114463656555
[ -7.56122449 -19.5       ]
0.5659114463656555
[ -6.76530612 -19.5       ]
0.5659114463656555
[ -5.96938776 -19.5       ]
0.5659114463656555
[ -5.17346939 -19.5       ]
0.5659114463656555
[ -4.37755102 -19.5       ]
0.5659114463656555
[ -3.58163265 -19.5       ]
0.5659114463656555
[ -2.78571429 -19.5       ]
0.5659114463656555
[ -1.98979592 -19.5       ]
0.565911446365655

0.5659114463656555
[-17.1122449  -16.31632653]
0.5659114463656555
[-16.31632653 -16.31632653]
0.5659114463656555
[-15.52040816 -16.31632653]
0.5659114463656555
[-14.7244898  -16.31632653]
0.5659114463656555
[-13.92857143 -16.31632653]
0.5659114463656555
[-13.13265306 -16.31632653]
0.5659114463656555
[-12.33673469 -16.31632653]
0.5659114463656555
[-11.54081633 -16.31632653]
0.5659114463656555
[-10.74489796 -16.31632653]
0.5659114463656555
[ -9.94897959 -16.31632653]
0.5659114463656555
[ -9.15306122 -16.31632653]
0.5659114463656555
[ -8.35714286 -16.31632653]
0.5659114463656555
[ -7.56122449 -16.31632653]
0.5659114463656555
[ -6.76530612 -16.31632653]
0.5659114463656555
[ -5.96938776 -16.31632653]
0.5659114463656555
[ -5.17346939 -16.31632653]
0.5659114463656555
[ -4.37755102 -16.31632653]
0.5659114463656555
[ -3.58163265 -16.31632653]
0.5659114463656555
[ -2.78571429 -16.31632653]
0.5659114463656555
[ -1.98979592 -16.31632653]
0.5659114463656555
[ -1.19387755 -16.31632653]
0.56591144636

[  2.78571429 -13.92857143]
0.5695593142041973
[  3.58163265 -13.92857143]
0.5695593142041973
[  4.37755102 -13.92857143]
0.5695593142041973
[  5.17346939 -13.92857143]
0.5695593142041973
[  5.96938776 -13.92857143]
0.5695593142041973
[  6.76530612 -13.92857143]
0.5695593142041973
[  7.56122449 -13.92857143]
0.5695593142041973
[  8.35714286 -13.92857143]
0.5695593142041973
[  9.15306122 -13.92857143]
0.5695593142041973
[  9.94897959 -13.92857143]
0.5695593142041973
[ 10.74489796 -13.92857143]
0.5695593142041973
[ 11.54081633 -13.92857143]
0.5695593142041973
[ 12.33673469 -13.92857143]
0.5695593142041973
[ 13.13265306 -13.92857143]
0.5695593142041973
[ 13.92857143 -13.92857143]
0.5695593142041973
[ 14.7244898  -13.92857143]
0.5695593142041973
[ 15.52040816 -13.92857143]
0.5695593142041973
[ 16.31632653 -13.92857143]
0.5695593142041973
[ 17.1122449  -13.92857143]
0.5695593142041973
[ 17.90816327 -13.92857143]
0.5695593142041973
[ 18.70408163 -13.92857143]
0.5695593142041973
[ 19.5       

...

0.5489236270680153
[6.76530612 5.17346939]
0.5489236270680153
[7.56122449 5.17346939]
0.5489236270680153
[8.35714286 5.17346939]
0.5489236270680153
[9.15306122 5.17346939]
0.5489236270680153
[9.94897959 5.17346939]
0.5489236270680153
[10.74489796  5.17346939]
0.5489236270680153
[11.54081633  5.17346939]
0.5489236270680153
[12.33673469  5.17346939]
0.5489236270680153
[13.13265306  5.17346939]
0.5489236270680153
[13.92857143  5.17346939]
0.5489236270680153
[14.7244898   5.17346939]
0.5489236270680153
[15.52040816  5.17346939]
0.5489236270680153
[16.31632653  5.17346939]
0.5489236270680153
[17.1122449   5.17346939]
0.5489236270680153
[17.90816327  5.17346939]
0.5489236270680153
[18.70408163  5.17346939]
0.5489236270680153
[19.5         5.17346939]
0.5489236270680153
[-19.5          5.96938776]
0.5489236270680153
[-18.70408163   5.96938776]
0.5489236270680153
[-17.90816327   5.96938776]
0.5489236270680153
[-17.1122449    5.96938776]
0.5489236270680153
[-16.31632653   5.96938776]
0.54892362

0.5735314462699141
[9.94897959 8.35714286]
0.5735314462699141
[10.74489796  8.35714286]
0.5735314462699141
[11.54081633  8.35714286]
0.5735314462699141
[12.33673469  8.35714286]
0.5735314462699141
[13.13265306  8.35714286]
0.5735314462699141
[13.92857143  8.35714286]
0.5735314462699141
[14.7244898   8.35714286]
0.5735314462699141
[15.52040816  8.35714286]
0.5735314462699141
[16.31632653  8.35714286]
0.5735314462699141
[17.1122449   8.35714286]
0.5735314462699141
[17.90816327  8.35714286]
0.5735314462699141
[18.70408163  8.35714286]
0.5735314462699141
[19.5         8.35714286]
0.5735314462699141
[-19.5          9.15306122]
0.5735314462699141
[-18.70408163   9.15306122]
0.5735314462699141
[-17.90816327   9.15306122]
0.5735314462699141
[-17.1122449    9.15306122]
0.5735314462699141
[-16.31632653   9.15306122]
0.5735314462699141
[-15.52040816   9.15306122]
0.5735314462699141
[-14.7244898    9.15306122]
0.5735314462699141
[-13.92857143   9.15306122]
0.5735314462699141
[-13.13265306   9.1530

0.5735314462699141
[15.52040816 11.54081633]
0.5735314462699141
[16.31632653 11.54081633]
0.5735314462699141
[17.1122449  11.54081633]
0.5735314462699141
[17.90816327 11.54081633]
0.5735314462699141
[18.70408163 11.54081633]
0.5735314462699141
[19.5        11.54081633]
0.5735314462699141
[-19.5         12.33673469]
0.564508774701287
[-18.70408163  12.33673469]
0.564508774701287
[-17.90816327  12.33673469]
0.564508774701287
[-17.1122449   12.33673469]
0.564508774701287
[-16.31632653  12.33673469]
0.564508774701287
[-15.52040816  12.33673469]
0.564508774701287
[-14.7244898   12.33673469]
0.564508774701287
[-13.92857143  12.33673469]
0.564508774701287
[-13.13265306  12.33673469]
0.564508774701287
[-12.33673469  12.33673469]
0.564508774701287
[-11.54081633  12.33673469]
0.564508774701287
[-10.74489796  12.33673469]
0.564508774701287
[-9.94897959 12.33673469]
0.564508774701287
[-9.15306122 12.33673469]
0.564508774701287
[-8.35714286 12.33673469]
0.564508774701287
[-7.56122449 12.33673469]
0

[ 9.15306122 14.7244898 ]
0.564508774701287
[ 9.94897959 14.7244898 ]
0.564508774701287
[10.74489796 14.7244898 ]
0.564508774701287
[11.54081633 14.7244898 ]
0.564508774701287
[12.33673469 14.7244898 ]
0.564508774701287
[13.13265306 14.7244898 ]
0.564508774701287
[13.92857143 14.7244898 ]
0.564508774701287
[14.7244898 14.7244898]
0.564508774701287
[15.52040816 14.7244898 ]
0.564508774701287
[16.31632653 14.7244898 ]
0.564508774701287
[17.1122449 14.7244898]
0.564508774701287
[17.90816327 14.7244898 ]
0.564508774701287
[18.70408163 14.7244898 ]
0.564508774701287
[19.5       14.7244898]
0.564508774701287
[-19.5         15.52040816]
0.564508774701287
[-18.70408163  15.52040816]
0.564508774701287
[-17.90816327  15.52040816]
0.564508774701287
[-17.1122449   15.52040816]
0.564508774701287
[-16.31632653  15.52040816]
0.564508774701287
[-15.52040816  15.52040816]
0.564508774701287
[-14.7244898   15.52040816]
0.564508774701287
[-13.92857143  15.52040816]
0.564508774701287
[-13.13265306  15.5204

0.5599550733379289
[ 6.76530612 17.90816327]
0.5599550733379289
[ 7.56122449 17.90816327]
0.5599550733379289
[ 8.35714286 17.90816327]
0.5599550733379289
[ 9.15306122 17.90816327]
0.5599550733379289
[ 9.94897959 17.90816327]
0.5599550733379289
[10.74489796 17.90816327]
0.5599550733379289
[11.54081633 17.90816327]
0.5599550733379289
[12.33673469 17.90816327]
0.5599550733379289
[13.13265306 17.90816327]
0.5599550733379289
[13.92857143 17.90816327]
0.5599550733379289
[14.7244898  17.90816327]
0.5599550733379289
[15.52040816 17.90816327]
0.5599550733379289
[16.31632653 17.90816327]
0.5599550733379289
[17.1122449  17.90816327]
0.5599550733379289
[17.90816327 17.90816327]
0.5599550733379289
[18.70408163 17.90816327]
0.5599550733379289
[19.5        17.90816327]
0.5599550733379289
[-19.5         18.70408163]
0.5599550733379289
[-18.70408163  18.70408163]
0.5599550733379289
[-17.90816327  18.70408163]
0.5599550733379289
[-17.1122449   18.70408163]
0.5599550733379289
[-16.31632653  18.70408163]


In [ ]:
## To generate a tfrecords file for encoder only (VIN) training

video_title = 'Video_ball_color_many_backs'
loadmat = np.load(video_title+'loadmat.npy')
params = np.load(video_title+'params.npy')

datadirectory = 'datadirectory/toydynamics_nograv/'+video_title+'_dynback.mp4'
extension = 'encoder_train.tfrecords'



def vids_to_encodertrain_eff_(datadirectory,extension,loadmat,params): ## assumes the ball is in the center 
    # We iterate through all directories
    videos = glob.glob(datadirectory)
    print(videos)
    for videoname in videos:
        print('working on video: ' +videoname)
        # Choose a video, and load it
        video = imageio.get_reader(videoname,'ffmpeg')
        # We want to open a corresponding tfRecords file
        datafile = videoname.split('.mp4')[0]
        print(datafile,'ler')
        datafile_name = datafile+extension
        number = int(datafile.split(video_title+'_b')[1])
        print(number,'her')
        writer = tf.python_io.TFRecordWriter(datafile_name)
        nb_frames = video.get_length()
        for i in range(nb_frames):
            # Iterate through frames
            example = _encoder_example_eff_(videoname,video,i,params[number,:])
            writer.write(example.SerializeToString())
        writer.close()
        sys.stdout.flush()
vids_to_encodertrain_eff_(datadirectory,extension,loadmat,params)

In [20]:
frames

array([     0,     83,    166, ..., 249832, 249915, 249999])